In [1]:
import os
import datetime
import argparse
import copy
import wandb
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data.distributed
import torch.utils.data

import utils.common as utils
from data import cifar10
from models.cifar10.vgg import vgg_16_bn
from models.cifar10.resnet import resnet_56
from decomposition.decomposition import decompose, decompose_conv_layer

In [2]:
# train_loader, val_loader = cifar10.load_data('../data', 256)
# criterion = nn.CrossEntropyLoss()
# criterion = criterion.cuda()
# cudnn.benchmark = True
# cudnn.enabled = True

# # load model
# compress_rate = utils.get_cpr('[0.]*100')

In [3]:
# model = eval('vgg_16_bn')(compress_rate=compress_rate).cuda()
# ckpt = torch.load('checkpoint/cifar10/vgg_16_bn.pt', map_location='cuda:0')
# model.load_state_dict(ckpt['state_dict'])

In [4]:
# dcp_model = decompose(model, rank=8, n_iter_max=100, n_iter_singular_error=1)

In [5]:
# def validate(val_loader, model, criterion):
#     losses = utils.AverageMeter('Loss', ':.4e')
#     top1 = utils.AverageMeter('Acc@1', ':6.2f')
#     top5 = utils.AverageMeter('Acc@5', ':6.2f')

#     # switch to evaluation mode
#     model.eval()
#     with torch.no_grad():
#         for i, (images, target) in enumerate(val_loader):
#             images = images.cuda()
#             target = target.cuda()

#             # compute output
#             logits = model(images)
#             loss = criterion(logits, target)

#             # measure accuracy and record loss
#             pred1, pred5 = utils.accuracy(logits, target, topk=(1, 5))
#             n = images.size(0)
#             losses.update(loss.item(), n)
#             top1.update(pred1[0], n)
#             top5.update(pred5[0], n)

#         print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
#                     .format(top1=top1, top5=top5))

#     return losses.avg, top1.avg, top5.avg


In [6]:
# model

In [7]:
# validate(val_loader, model, criterion)

In [8]:
conv = nn.Conv2d(64, 128, 3, padding=1, bias=False).cuda()

In [9]:
dcp_layer = decompose_conv_layer(conv, rank=9, n_iter_max=100)

  0%|          | 0/128 [00:00<?, ?it/s]

In [10]:
dcp_layer

CPDBlock(
  (head): ModuleList(
    (0-127): 128 x Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
  (body): ModuleList(
    (0-127): 128 x Conv2d(8, 1, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
  )
  (tail): ModuleList(
    (0-127): 128 x Conv2d(1, 8, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
  )
)

In [11]:
def nmse(conv, dcp):
  rand_inp = torch.randn((1, 64, 32, 32), device=0)
  ori_output = conv(rand_inp)
  my_output = dcp(rand_inp)
  error = (torch.norm(ori_output-my_output, p=2).item()/torch.norm(ori_output, p=2).item())**2
  
  return error

In [12]:
print(nmse(conv, dcp_layer))

self.head[0].weight.shape  torch.Size([8, 64, 1, 1])
self.body[0].weight.shape  torch.Size([1, 8, 3, 1])
self.tail[0].weight.shape  torch.Size([8, 1, 1, 3])
4768.592741641466
